In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
print(tf.__version__)
print(keras.__version__)
devicelist = tf.config.list_physical_devices()
print(devicelist[2])

2.3.0
2.4.0
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
df = pd.read_csv('C:/Users/Kevin/Desktop/SPH6004 Assignment 2/assignment2_data/mimiciv_aki/time_series.csv')

pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 2)
df

,Time,stay_id,hadm_id,age,gender,Heart Rate,Respiratory Rate,SpO2/SaO2,pH,Potassium,Calcium,Glucose,Sodium,HCO3,White Blood Cells,Hemoglobin,Red Blood Cells,Platelet Count,Weight,Urea Nitrogen,Creatinine,Blood Pressure,1 hours urine output,6 hours urine output,Next 6 hours urine output,AKI,gcs,ventilation,vasoactive medications,sedative medications
0,2111-09-04 04:00:00,33225641,27845175,32,0,104.0,21.0,97.0,NaN,4.4,8.4,113.0,136.0,19.0,18.7,9.8,3.25,185.0,NaN,7.0,0.4,99.0,0.0,0.0,925.0,0,15.0,0,0,1
1,2111-09-04 05:00:00,33225641,27845175,32,0,104.0,25.0,95.0,7.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,550.0,0.0,995.0,0,NaN,0,0,1
2,2111-09-04 06:00:00,33225641,27845175,32,0,120.0,18.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.0,0.0,0.0,445.0,0,NaN,0,0,0
3,2111-09-04 07:00:00,33225641,27845175,32,0,106.0,18.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,275.0,0.0,520.0,0,NaN,0,0,1
4,2111-09-04 08:00:00,33225641,27845175,32,0,103.0,17.5,98.0,7.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,0.0,0.0,245.0,0,15.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3969967,2152-03-03 12:00:00,30591465,29623448,19,1,96.0,20.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.0,80.0,480.0,390.0,0,15.0,0,0,1
3969968,2152-03-03 13:00:00,30591465,29623448,19,1,72.0,24.0,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,100.0,500.0,355.0,0,NaN,0,0,1
3969969,2152-03-03 14:00:00,30591465,29623448,19,1,71.0,20.0,97.0,NaN,NaN,NaN,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0,30.0,600.0,300.0,0,NaN,0,0,1
3969970,2152-03-03 15:00:00,30591465,29623448,19,1,82.0,17.0,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,40.0,510.0,330.0,0,NaN,0,0,1


In [4]:
## Evaluate whether 'stay_id' or 'hadm_id' is the unique identifier.
print(df['stay_id'].value_counts(), '\n')
print(df['hadm_id'].value_counts(), '\n')
print(df.loc[df['hadm_id']==24748146, ['stay_id']].value_counts(), '\n')
print("A single hadm_id can have multiple stay_ids. Therefore, stay_id is the unique identifier. Longest stay was 715 timesteps.")

37805052    715
36043022    714
39374669    714
39579609    714
37149012    714
           ... 
38023369      7
36564267      7
39163397      7
31990580      7
35699916      7
Name: stay_id, Length: 39742, dtype: int64 

24748146    1968
29281474    1891
25963110    1880
26488509    1738
22793142    1556
            ... 
24088081       7
25395928       7
22344424       7
25248779       7
25854706       7
Name: hadm_id, Length: 36578, dtype: int64 

stay_id 
31505890    599
30962383    594
32142854    586
36616293    167
34838186     22
dtype: int64 

A single hadm_id can have multiple stay_ids. Therefore, stay_id is the unique identifier. Longest stay was 715 timesteps.


In [5]:
## Create backup of 'stay_id'.
df['stay_id_backup'] = df['stay_id']

## Create a list of unique 'stay_id's.
stay_id_list = df['stay_id'].unique().tolist()
print('Length of stay_id list =', len(stay_id_list))

## Randomly select 4000 stay_ids (~10%) to be test set.
rng = np.random.default_rng(123456789)
stay_id_test = rng.choice(stay_id_list, size=4000).tolist()
print('Length of stay_id_test =', len(stay_id_test))

## Return stay_id training set.
stay_id_train = list(set(stay_id_list)-set(stay_id_test))
print('Length of stay_id_train =', len(stay_id_train))

Length of stay_id list = 39742
Length of stay_id_test = 4000
Length of stay_id_train = 35962


In [6]:
## Fix date time column.
df['Time'] = pd.to_datetime(df['Time'])

## For each stay_id, get date & time of first record.
in_time = df.groupby('stay_id')['Time'].first()

## Merge 'in_time' back into main dataframe.
df = df.merge(in_time, how='inner', on='stay_id')

## For each timestep in each stay_id, calculate hours elapsed since first record.
df['hours_elapsed'] = (df['Time_x'] - df['Time_y']) / np.timedelta64(1, 'h')
df['hours_elapsed'] = df['hours_elapsed'].astype(int)
df

,Time_x,stay_id,hadm_id,age,gender,Heart Rate,Respiratory Rate,SpO2/SaO2,pH,Potassium,Calcium,Glucose,Sodium,HCO3,White Blood Cells,Hemoglobin,Red Blood Cells,Platelet Count,Weight,Urea Nitrogen,Creatinine,Blood Pressure,1 hours urine output,6 hours urine output,Next 6 hours urine output,AKI,gcs,ventilation,vasoactive medications,sedative medications,stay_id_backup,Time_y,hours_elapsed
0,2111-09-04 04:00:00,33225641,27845175,32,0,104.0,21.0,97.0,NaN,4.4,8.4,113.0,136.0,19.0,18.7,9.8,3.25,185.0,NaN,7.0,0.4,99.0,0.0,0.0,925.0,0,15.0,0,0,1,33225641,2111-09-04 04:00:00,0
1,2111-09-04 05:00:00,33225641,27845175,32,0,104.0,25.0,95.0,7.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,550.0,0.0,995.0,0,NaN,0,0,1,33225641,2111-09-04 04:00:00,1
2,2111-09-04 06:00:00,33225641,27845175,32,0,120.0,18.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.0,0.0,0.0,445.0,0,NaN,0,0,0,33225641,2111-09-04 04:00:00,2
3,2111-09-04 07:00:00,33225641,27845175,32,0,106.0,18.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,275.0,0.0,520.0,0,NaN,0,0,1,33225641,2111-09-04 04:00:00,3
4,2111-09-04 08:00:00,33225641,27845175,32,0,103.0,17.5,98.0,7.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,0.0,0.0,245.0,0,15.0,0,0,1,33225641,2111-09-04 04:00:00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3969967,2152-03-03 12:00:00,30591465,29623448,19,1,96.0,20.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.0,80.0,480.0,390.0,0,15.0,0,0,1,30591465,2152-02-20 10:00:00,290
3969968,2152-03-03 13:00:00,30591465,29623448,19,1,72.0,24.0,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,100.0,500.0,355.0,0,NaN,0,0,1,30591465,2152-02-20 10:00:00,291
3969969,2152-03-03 14:00:00,30591465,29623448,19,1,71.0,20.0,97.0,NaN,NaN,NaN,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0,30.0,600.0,300.0,0,NaN,0,0,1,30591465,2152-02-20 10:00:00,292
3969970,2152-03-03 15:00:00,30591465,29623448,19,1,82.0,17.0,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,40.0,510.0,330.0,0,NaN,0,0,1,30591465,2152-02-20 10:00:00,293


In [7]:
## For each stay_id, timestep = hours elapsed since first record.
df['timestep'] = df['hours_elapsed']

## Set the rows of df to contain a multi-level index: first level = stay_id, second level = timestep.
df = df.set_index(['stay_id', 'timestep'])
df

Time_x   hadm_id  age  gender  Heart Rate  \
stay_id  timestep                                                          
33225641 0        2111-09-04 04:00:00  27845175   32       0       104.0   
         1        2111-09-04 05:00:00  27845175   32       0       104.0   
         2        2111-09-04 06:00:00  27845175   32       0       120.0   
         3        2111-09-04 07:00:00  27845175   32       0       106.0   
         4        2111-09-04 08:00:00  27845175   32       0       103.0   
...                               ...       ...  ...     ...         ...   
30591465 290      2152-03-03 12:00:00  29623448   19       1        96.0   
         291      2152-03-03 13:00:00  29623448   19       1        72.0   
         292      2152-03-03 14:00:00  29623448   19       1        71.0   
         293      2152-03-03 15:00:00  29623448   19       1        82.0   
         294      2152-03-03 16:00:00  29623448   19       1        90.0   

                   Respiratory Rate  SpO2/SaO2    pH  Potassium  Calcium  \
stay_id  timestep                                                          
33225641 0                     21.0       97.0   NaN        4.4      8.4   
         1                     25.0       95.0  7.38        NaN      NaN   
         2                     18.0       98.0   NaN        NaN      NaN   
         3                     18.0       98.0   NaN        NaN      NaN   
         4                     17.5       98.0  7.28        NaN      NaN   
...                             ...        ...   ...        ...      ...   
30591465 290                   20.0       98.0   NaN        NaN      NaN   
         291                   24.0       97.0   NaN        NaN      NaN   
         292                   20.0       97.0   NaN        NaN      NaN   
         293                   17.0       97.0   NaN        NaN      NaN   
         294                   20.0      100.0   NaN        NaN      NaN   

                   Glucose  Sodium  HCO3  White Blood Cells  Hemoglobin  \
stay_id  timestep                                                         
33225641 0           113.0   136.0  19.0               18.7         9.8   
         1             NaN     NaN   NaN                NaN         NaN   
         2             NaN     NaN   NaN                NaN         NaN   
         3             NaN     NaN   NaN                NaN         NaN   
         4             NaN     NaN   NaN                NaN         NaN   
...                    ...     ...   ...                ...         ...   
30591465 290           NaN     NaN   NaN                NaN         NaN   
         291           NaN     NaN   NaN                NaN         NaN   
         292         139.0     NaN   NaN                NaN         NaN   
         293           NaN     NaN   NaN                NaN         NaN   
         294           NaN     NaN   NaN                NaN         NaN   

                   Red Blood Cells  Platelet Count  Weight  Urea Nitrogen  \
stay_id  timestep                                                           
33225641 0                    3.25           185.0     NaN            7.0   
         1                     NaN             NaN     NaN            NaN   
         2                     NaN             NaN     NaN            NaN   
         3                     NaN             NaN     NaN            NaN   
         4                     NaN             NaN     NaN            NaN   
...                            ...             ...     ...            ...   
30591465 290                   NaN             NaN     NaN            NaN   
         291                   NaN             NaN     NaN            NaN   
         292                   NaN             NaN     NaN            NaN   
         293                   NaN             NaN     NaN            NaN   
         294                   NaN             NaN     NaN            NaN   

                   Creatinine  Blood Pressure  1 hours urine output  \
stay

In [8]:
## For each stay_id, fill missing values for each feature in timestep t with values from timestep t-1. Failing which, fill missing values from timestep t+1 instead.
df = df.groupby('stay_id').fillna(method='ffill').fillna(method='bfill')
df

Time_x   hadm_id  age  gender  Heart Rate  \
stay_id  timestep                                                          
33225641 0        2111-09-04 04:00:00  27845175   32       0       104.0   
         1        2111-09-04 05:00:00  27845175   32       0       104.0   
         2        2111-09-04 06:00:00  27845175   32       0       120.0   
         3        2111-09-04 07:00:00  27845175   32       0       106.0   
         4        2111-09-04 08:00:00  27845175   32       0       103.0   
...                               ...       ...  ...     ...         ...   
30591465 290      2152-03-03 12:00:00  29623448   19       1        96.0   
         291      2152-03-03 13:00:00  29623448   19       1        72.0   
         292      2152-03-03 14:00:00  29623448   19       1        71.0   
         293      2152-03-03 15:00:00  29623448   19       1        82.0   
         294      2152-03-03 16:00:00  29623448   19       1        90.0   

                   Respiratory Rate  SpO2/SaO2    pH  Potassium  Calcium  \
stay_id  timestep                                                          
33225641 0                     21.0       97.0  7.38        4.4      8.4   
         1                     25.0       95.0  7.38        4.4      8.4   
         2                     18.0       98.0  7.38        4.4      8.4   
         3                     18.0       98.0  7.38        4.4      8.4   
         4                     17.5       98.0  7.28        4.4      8.4   
...                             ...        ...   ...        ...      ...   
30591465 290                   20.0       98.0  7.44        4.0      8.8   
         291                   24.0       97.0  7.44        4.0      8.8   
         292                   20.0       97.0  7.44        4.0      8.8   
         293                   17.0       97.0  7.44        4.0      8.8   
         294                   20.0      100.0  7.44        4.0      8.8   

                   Glucose  Sodium  HCO3  White Blood Cells  Hemoglobin  \
stay_id  timestep                                                         
33225641 0           113.0   136.0  19.0               18.7         9.8   
         1           113.0   136.0  19.0               18.7         9.8   
         2           113.0   136.0  19.0               18.7         9.8   
         3           113.0   136.0  19.0               18.7         9.8   
         4           113.0   136.0  19.0               18.7         9.8   
...                    ...     ...   ...                ...         ...   
30591465 290         131.0   152.0  29.0                8.8         9.5   
         291         131.0   152.0  29.0                8.8         9.5   
         292         139.0   152.0  29.0                8.8         9.5   
         293         139.0   152.0  29.0                8.8         9.5   
         294         139.0   152.0  29.0                8.8         9.5   

                   Red Blood Cells  Platelet Count  Weight  Urea Nitrogen  \
stay_id  timestep                                                           
33225641 0                    3.25           185.0   104.3            7.0   
         1                    3.25           185.0   104.3            7.0   
         2                    3.25           185.0   104.3            7.0   
         3                    3.25           185.0   104.3            7.0   
         4                    3.25           185.0   104.3            7.0   
...                            ...             ...     ...            ...   
30591465 290                  3.11           528.0    95.8           22.0   
         291                  3.11           528.0    95.8           22.0   
         292                  3.11           528.0    95.8           22.0   
         293                  3.11           528.0    95.8           22.0   
         294                  3.11           528.0    95.8           22.0   

                   Creatinine  Blood Pressure  1 hours urine output  \
stay

In [9]:
## Drop 'Time_x', hadm_id', 'Next 6 hours urine output', and 'Time_y' variables.
df = df.drop(columns=['Time_x', 'hadm_id', 'Next 6 hours urine output', 'Time_y'])
df

age  gender  Heart Rate  Respiratory Rate  SpO2/SaO2    pH  \
stay_id  timestep                                                               
33225641 0          32       0       104.0              21.0       97.0  7.38   
         1          32       0       104.0              25.0       95.0  7.38   
         2          32       0       120.0              18.0       98.0  7.38   
         3          32       0       106.0              18.0       98.0  7.38   
         4          32       0       103.0              17.5       98.0  7.28   
...                ...     ...         ...               ...        ...   ...   
30591465 290        19       1        96.0              20.0       98.0  7.44   
         291        19       1        72.0              24.0       97.0  7.44   
         292        19       1        71.0              20.0       97.0  7.44   
         293        19       1        82.0              17.0       97.0  7.44   
         294        19       1        90.0              20.0      100.0  7.44   

                   Potassium  Calcium  Glucose  Sodium  HCO3  \
stay_id  timestep                                              
33225641 0               4.4      8.4    113.0   136.0  19.0   
         1               4.4      8.4    113.0   136.0  19.0   
         2               4.4      8.4    113.0   136.0  19.0   
         3               4.4      8.4    113.0   136.0  19.0   
         4               4.4      8.4    113.0   136.0  19.0   
...                      ...      ...      ...     ...   ...   
30591465 290             4.0      8.8    131.0   152.0  29.0   
         291             4.0      8.8    131.0   152.0  29.0   
         292             4.0      8.8    139.0   152.0  29.0   
         293             4.0      8.8    139.0   152.0  29.0   
         294             4.0      8.8    139.0   152.0  29.0   

                   White Blood Cells  Hemoglobin  Red Blood Cells  \
stay_id  timestep                                                   
33225641 0                      18.7         9.8             3.25   
         1                      18.7         9.8             3.25   
         2                      18.7         9.8             3.25   
         3                      18.7         9.8             3.25   
         4                      18.7         9.8             3.25   
...                              ...         ...              ...   
30591465 290                     8.8         9.5             3.11   
         291                     8.8         9.5             3.11   
         292                     8.8         9.5             3.11   
         293                     8.8         9.5             3.11   
         294                     8.8         9.5             3.11   

                   Platelet Count  Weight  Urea Nitrogen  Creatinine  \
stay_id  timestep                                                      
33225641 0                  185.0   104.3            7.0         0.4   
         1                  185.0   104.3            7.0         0.4   
         2                  185.0   104.3            7.0         0.4   
         3                  185.0   104.3            7.0         0.4   
         4                  185.0   104.3            7.0         0.4   
...                           ...     ...            ...         ...   
30591465 290                528.0    95.8           22.0         0.8   
         291                528.0    95.8           22.0         0.8   
         292                528.0    95.8           22.0         0.8   
         293                528.0    95.8           22.0         0.8   
         294                528.0    95.8           22.0         0.8   

                   Blood Pressure  1 hours urine output  6 hours urine output  \
stay_id  timestep                                                               
33225641 0                   99.0                   0.0                   0.0   
         1                   86.0                 550.

In [10]:
## For each stay_id, at each timestep, generate the AKI variable 'future_hours' into the future. Fill newly introduced missing values with np.nan.
future_hours = 6
df['AKI_future'] = df.groupby('stay_id')['AKI'].shift(periods=-(future_hours), fill_value=np.nan)

## Check execution on some examples.
example = df.loc[stay_id_list[3]]
print(example.loc[example['AKI_future']==1, ['AKI', 'AKI_future']])
print(example.loc[example['AKI']==1, ['AKI', 'AKI_future']])
example2 = df.loc[stay_id_list[-1]]
print(example2.loc[example2['AKI_future']==1, ['AKI', 'AKI_future']])
print(example2.loc[example2['AKI']==1, ['AKI', 'AKI_future']])

## For each stay_id, drop the last timesteps where 'AKI_future' is unknown.
df = df.dropna(axis=0, how='any')

## Drop 'stay_id_backup'.
df = df.drop(columns='stay_id_backup')
df

          AKI  AKI_future
timestep                 
80          0         1.0
          AKI  AKI_future
timestep                 
86          1         0.0
          AKI  AKI_future
timestep                 
56          0         1.0
          AKI  AKI_future
timestep                 
62          1         0.0


age  gender  Heart Rate  Respiratory Rate  SpO2/SaO2    pH  \
stay_id  timestep                                                               
33225641 0          32       0       104.0              21.0       97.0  7.38   
         1          32       0       104.0              25.0       95.0  7.38   
         2          32       0       120.0              18.0       98.0  7.38   
         3          32       0       106.0              18.0       98.0  7.38   
         4          32       0       103.0              17.5       98.0  7.28   
...                ...     ...         ...               ...        ...   ...   
30591465 284        19       1        99.0              23.0       99.0  7.44   
         285        19       1        99.0              19.0       99.0  7.44   
         286        19       1        95.0              25.0       99.0  7.44   
         287        19       1        90.0              22.0       97.0  7.44   
         288        19       1        67.0              18.0       97.0  7.44   

                   Potassium  Calcium  Glucose  Sodium  HCO3  \
stay_id  timestep                                              
33225641 0               4.4      8.4    113.0   136.0  19.0   
         1               4.4      8.4    113.0   136.0  19.0   
         2               4.4      8.4    113.0   136.0  19.0   
         3               4.4      8.4    113.0   136.0  19.0   
         4               4.4      8.4    113.0   136.0  19.0   
...                      ...      ...      ...     ...   ...   
30591465 284             4.0      8.8    120.5   152.0  29.0   
         285             4.0      8.8    120.5   152.0  29.0   
         286             4.0      8.8    131.0   152.0  29.0   
         287             4.0      8.8    131.0   152.0  29.0   
         288             4.0      8.8    131.0   152.0  29.0   

                   White Blood Cells  Hemoglobin  Red Blood Cells  \
stay_id  timestep                                                   
33225641 0                      18.7         9.8             3.25   
         1                      18.7         9.8             3.25   
         2                      18.7         9.8             3.25   
         3                      18.7         9.8             3.25   
         4                      18.7         9.8             3.25   
...                              ...         ...              ...   
30591465 284                     8.8         9.5             3.11   
         285                     8.8         9.5             3.11   
         286                     8.8         9.5             3.11   
         287                     8.8         9.5             3.11   
         288                     8.8         9.5             3.11   

                   Platelet Count  Weight  Urea Nitrogen  Creatinine  \
stay_id  timestep                                                      
33225641 0                  185.0   104.3            7.0         0.4   
         1                  185.0   104.3            7.0         0.4   
         2                  185.0   104.3            7.0         0.4   
         3                  185.0   104.3            7.0         0.4   
         4                  185.0   104.3            7.0         0.4   
...                           ...     ...            ...         ...   
30591465 284                528.0    95.8           22.0         0.8   
         285                528.0    95.8           22.0         0.8   
         286                528.0    95.8           22.0         0.8   
         287                528.0    95.8           22.0         0.8   
         288                528.0    95.8           22.0         0.8   

                   Blood Pressure  1 hours urine output  6 hours urine output  \
stay_id  timestep                                                               
33225641 0                   99.0                   0.0                   0.0   
         1                   86.0                 550.

In [11]:
## Convert this multi-level indexed dataframe into an xarray dataset with 2 dimensions (stay_id, timestep) and 28 data variables.
## Dataset is automatically aligned along the timestep dimension, padding all stay_ids to the maximum timestep length.
xds = df.to_xarray()
print(xds.sizes)
xds

Frozen(SortedKeysDict({'stay_id': 39742, 'timestep': 709}))


<xarray.Dataset>
Dimensions:                 (stay_id: 39742, timestep: 709)
Coordinates:
  * stay_id                 (stay_id) int64 30000010 30000575 ... 39998706
  * timestep                (timestep) int64 0 1 2 3 4 5 ... 704 705 706 707 708
Data variables: (12/28)
    age                     (stay_id, timestep) float64 71.0 71.0 ... nan nan
    gender                  (stay_id, timestep) float64 1.0 1.0 1.0 ... nan nan
    Heart Rate              (stay_id, timestep) float64 76.6 76.6 ... nan nan
    Respiratory Rate        (stay_id, timestep) float64 19.0 19.0 ... nan nan
    SpO2/SaO2               (stay_id, timestep) float64 97.0 97.0 ... nan nan
    pH                      (stay_id, timestep) float64 7.3 7.36 ... nan nan
    ...                      ...
    gcs                     (stay_id, timestep) float64 3.0 3.0 3.0 ... nan nan
    ventilation             (stay_id, timestep) float64 0.0 0.0 0.0 ... nan nan
    vasoactive medications  (stay_id, timestep) float64 0.0 0.0 0.0 ... nan nan
    sedative medications    (stay_id, timestep) float64 0.0 0.0 0.0 ... nan nan
    hours_elapsed           (stay_id, timestep) float64 0.0 1.0 2.0 ... nan nan
    AKI_future              (stay_id, timestep) float64 0.0 0.0 0.0 ... nan nan

In [12]:
## Convert all padded values from NaN to 0.
xds = xds.fillna(0)
xds

<xarray.Dataset>
Dimensions:                 (stay_id: 39742, timestep: 709)
Coordinates:
  * stay_id                 (stay_id) int64 30000010 30000575 ... 39998706
  * timestep                (timestep) int64 0 1 2 3 4 5 ... 704 705 706 707 708
Data variables: (12/28)
    age                     (stay_id, timestep) float64 71.0 71.0 ... 0.0 0.0
    gender                  (stay_id, timestep) float64 1.0 1.0 1.0 ... 0.0 0.0
    Heart Rate              (stay_id, timestep) float64 76.6 76.6 ... 0.0 0.0
    Respiratory Rate        (stay_id, timestep) float64 19.0 19.0 ... 0.0 0.0
    SpO2/SaO2               (stay_id, timestep) float64 97.0 97.0 ... 0.0 0.0
    pH                      (stay_id, timestep) float64 7.3 7.36 ... 0.0 0.0
    ...                      ...
    gcs                     (stay_id, timestep) float64 3.0 3.0 3.0 ... 0.0 0.0
    ventilation             (stay_id, timestep) float64 0.0 0.0 0.0 ... 0.0 0.0
    vasoactive medications  (stay_id, timestep) float64 0.0 0.0 0.0 ... 0.0 0.0
    sedative medications    (stay_id, timestep) float64 0.0 0.0 0.0 ... 0.0 0.0
    hours_elapsed           (stay_id, timestep) float64 0.0 1.0 2.0 ... 0.0 0.0
    AKI_future              (stay_id, timestep) float64 0.0 0.0 0.0 ... 0.0 0.0

In [13]:
## Separate xds dataset into X_ds (features) and Y_ds (labels).
X_ds = xds.drop_vars('AKI_future')
Y_ds = xds['AKI_future']
print(X_ds, '\n \n', Y_ds)

<xarray.Dataset>
Dimensions:                 (stay_id: 39742, timestep: 709)
Coordinates:
  * stay_id                 (stay_id) int64 30000010 30000575 ... 39998706
  * timestep                (timestep) int64 0 1 2 3 4 5 ... 704 705 706 707 708
Data variables: (12/27)
    age                     (stay_id, timestep) float64 71.0 71.0 ... 0.0 0.0
    gender                  (stay_id, timestep) float64 1.0 1.0 1.0 ... 0.0 0.0
    Heart Rate              (stay_id, timestep) float64 76.6 76.6 ... 0.0 0.0
    Respiratory Rate        (stay_id, timestep) float64 19.0 19.0 ... 0.0 0.0
    SpO2/SaO2               (stay_id, timestep) float64 97.0 97.0 ... 0.0 0.0
    pH                      (stay_id, timestep) float64 7.3 7.36 ... 0.0 0.0
    ...                      ...
    AKI                     (stay_id, timestep) float64 1.0 0.0 0.0 ... 0.0 0.0
    gcs                     (stay_id, timestep) float64 3.0 3.0 3.0 ... 0.0 0.0
    ventilation             (stay_id, timestep) float64 0.0 0.0 0.0 

In [14]:
## Separate X_ds and Y_ds datasets into training and testing sets based on 'stay_id_train' and 'stay_id_test' lists generated earlier.
X_train = X_ds.sel(stay_id=stay_id_train)
X_test = X_ds.sel(stay_id=stay_id_test)

Y_train = Y_ds.sel(stay_id=stay_id_train)
Y_test = Y_ds.sel(stay_id=stay_id_test)

print(X_train, '\n \n', Y_train, '\n \n', X_test, '\n \n', Y_test)

<xarray.Dataset>
Dimensions:                 (stay_id: 35962, timestep: 709)
Coordinates:
  * stay_id                 (stay_id) int64 32243713 37224453 ... 38273020
  * timestep                (timestep) int64 0 1 2 3 4 5 ... 704 705 706 707 708
Data variables: (12/27)
    age                     (stay_id, timestep) float64 77.0 77.0 ... 0.0 0.0
    gender                  (stay_id, timestep) float64 1.0 1.0 1.0 ... 0.0 0.0
    Heart Rate              (stay_id, timestep) float64 98.33 98.33 ... 0.0 0.0
    Respiratory Rate        (stay_id, timestep) float64 15.25 15.25 ... 0.0 0.0
    SpO2/SaO2               (stay_id, timestep) float64 100.0 100.0 ... 0.0 0.0
    pH                      (stay_id, timestep) float64 7.42 7.3 7.3 ... 0.0 0.0
    ...                      ...
    AKI                     (stay_id, timestep) float64 0.0 0.0 0.0 ... 0.0 0.0
    gcs                     (stay_id, timestep) float64 15.0 15.0 ... 0.0 0.0
    ventilation             (stay_id, timestep) float64 0.0 

In [15]:
## Convert xarray datasets and dataarrays into numpy arrays. 3D numpy arrays for X datasets; 2D numpy arrays for Y dataarrays.
X_train_np = X_train.to_array().values
X_test_np = X_test.to_array().values

Y_train_np = Y_train.values
Y_test_np = Y_test.values

print('X_train_np shape:', X_train_np.shape)
print('X_test_np shape:', X_test_np.shape)
print('Y_train_np shape:', Y_train_np.shape)
print('Y_test_np shape:', Y_test_np.shape)

X_train_np shape: (27, 35962, 709)
X_test_np shape: (27, 4000, 709)
Y_train_np shape: (35962, 709)
Y_test_np shape: (4000, 709)


In [16]:
## Transpose the dimensions of the feature numpy arrays (X) for compatibility with Tensorflow input dimensions (batch, timesteps, feature)
X_train_np = X_train_np.transpose(1, 2, 0)
X_test_np = X_test_np.transpose(1, 2, 0)

print('X_train_np shape:', X_train_np.shape)
print('Y_train_np shape:', Y_train_np.shape)
print('X_test_np shape:', X_test_np.shape)
print('Y_test_np shape:', Y_test_np.shape)

X_train_np shape: (35962, 709, 27)
Y_train_np shape: (35962, 709)
X_test_np shape: (4000, 709, 27)
Y_test_np shape: (4000, 709)


In [17]:
## Create LSTM network. Layers: Input --> LSTM --> LSTM --> LSTM --> Dense --> Output (Probability of AKI in 'future_hours' hours).
X_input = keras.Input(shape=(X_train_np.shape[1], X_train_np.shape[2]))

X_next = layers.LSTM(128, return_sequences=True)(X_input)
X_next = layers.LSTM(128, return_sequences=True)(X_next)
X_next = layers.LSTM(128, return_sequences=True)(X_next)
X_next = layers.TimeDistributed(layers.Dense(32, activation='relu'))(X_next)

X_output = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(X_next)

model = keras.Model(inputs=X_input, outputs=X_output)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 709, 27)]         0         
_________________________________________________________________
lstm (LSTM)                  (None, 709, 128)          79872     
_________________________________________________________________
lstm_1 (LSTM)                (None, 709, 128)          131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 709, 128)          131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 709, 32)           4128      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 709, 1)            33        
Total params: 347,201
Trainable params: 347,201
Non-trainable params: 0
________________________________________________

In [18]:
## Define optimizer, loss function, and evaluation metrics.
adam = keras.optimizers.Adam()
bceloss = keras.losses.BinaryCrossentropy()
auc = keras.metrics.AUC()
model.compile(optimizer=adam, loss=bceloss, metrics=[auc])

In [19]:
## Train model on 10 epochs of the training data.
model.fit(x=X_train_np, y=Y_train_np, epochs=10)

Epoch 1/10
1124/1124 [==============================] - 131s 117ms/step - loss: 0.0128 - auc: 0.9083
Epoch 2/10
1124/1124 [==============================] - 195s 173ms/step - loss: 0.0088 - auc: 0.9385
Epoch 3/10
1124/1124 [==============================] - 193s 172ms/step - loss: 0.0085 - auc: 0.9385
Epoch 4/10
1124/1124 [==============================] - 139s 124ms/step - loss: 0.0082 - auc: 0.9422
Epoch 5/10
1124/1124 [==============================] - 136s 121ms/step - loss: 0.0081 - auc: 0.9411
Epoch 6/10
1124/1124 [==============================] - 139s 124ms/step - loss: 0.0081 - auc: 0.9414
Epoch 7/10
1124/1124 [==============================] - 143s 127ms/step - loss: 0.0080 - auc: 0.9405
Epoch 8/10
1124/1124 [==============================] - 137s 122ms/step - loss: 0.0077 - auc: 0.9422
Epoch 9/10
1124/1124 [==============================] - 136s 121ms/step - loss: 0.0078 - auc: 0.9424
Epoch 10/10
1124/1124 [==============================] - 138s 123ms/step - loss: 0.0077 - a

In [20]:
## Evaluate the performance of the model on the held-out test set (10% of the stay_ids).
model.evaluate(x=X_test_np, y=Y_test_np)

125/125 [==============================] - 7s 55ms/step - loss: 0.0077 - auc: 0.9548


[0.007659998722374439, 0.9547553062438965]

In [22]:
## Save the trained LSTM model.
model.save('AKI_LSTM_v7')

INFO:tensorflow:Assets written to: AKI_LSTM_v7\assets
